In [ ]:
import os
import json
import time
from cryptography.fernet import Fernet

# Generate encryption keys for each component
key_database = {
    "client": Fernet.generate_key(),
    "AS": Fernet.generate_key(),
    "TGS": Fernet.generate_key(),
    "service": Fernet.generate_key()
}

# ==========================
# 🔐 Encryption & Decryption
# ==========================

def encrypt_data(key, data):
    """Encrypts a JSON object using Fernet symmetric encryption."""
    # TODO: Implement encryption using Fernet
    pass  # Replace with actual encryption logic

def decrypt_data(key, encrypted_data):
    """Decrypts a JSON object using Fernet symmetric encryption."""
    # TODO: Implement decryption using Fernet
    pass  # Replace with actual decryption logic

# ==========================
# 🎟️ Authentication Server (AS) - Issues Ticket Granting Ticket (TGT)
# ==========================

def authentication_server(client_id):
    """Simulates the Authentication Server issuing a TGT to the client."""
    # TODO: Generate a session key and a nonce
    session_key = None
    nonce = None

    # TODO: Define what a TGT is

    # TODO: Encrypt the TGT with the AS key before returning
    encrypted_tgt = None  # Replace with encryption function

    return encrypted_tgt, session_key

# ==========================
# 🎫 Ticket Granting Server (TGS) - Issues Service Ticket
# ==========================

def ticket_granting_server(encrypted_tgt, client_session_key):
    """Simulates the Ticket Granting Server issuing a service ticket."""
    try:
        # TODO: Decrypt the TGT using AS's key
        decrypted_tgt = None  # Replace with decryption function

        # TODO: Validate ticket expiration (reject if > 5 minutes old)

        # TODO: Generate a session key and a nonce
        service_session_key = None
        nonce = None

        # TODO: Define a Service Ticket

        # TODO: Encrypt service ticket with TGS key before returning
        encrypted_service_ticket = None  # Replace with encryption function

        return encrypted_service_ticket, service_session_key

    except Exception as e:
        return f"❌ Error in TGS: {str(e)}", None

# ==========================
# 🏛️ Service Server - Grants Access
# ==========================

def service_server(encrypted_service_ticket):
    """Validates and decrypts the service ticket before granting access."""
    try:
        # TODO: Decrypt service ticket using TGS's key
        decrypted_ticket = None  # Replace with decryption function

        # TODO: Validate timestamp (Reject if expired)

        # TODO: Return info extracted from the ticket
        return f"✅ Access granted to [client ID] with key [session key]"

    except Exception as e:
        return f"❌ Service Authentication Failed: {str(e)}"

# ==========================
# 🖥️ Interactive CLI
# ==========================

def main():
    """Interactive command-line interface for running Kerberos authentication steps."""
    client_id = "client"
    tgt = None
    client_session_key = None
    service_ticket = None
    service_session_key = None

    while True:
        print("\n===== Kerberos Authentication System =====")
        print("1. Request Ticket Granting Ticket (TGT)")
        print("2. Request Service Ticket")
        print("3. Access Protected Service")
        print("4. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == "1":
            # TODO get the TGT and client session key, print both
            print(f"\n🔐 Received ... ")

        elif choice == "2":
            # TODO get the service ticket and service session key, print both
            print(f"\n🎟️ Received ... ")

        elif choice == "3":
            # TODO access the service, print the result
            print(f"\n Received ... ")

        elif choice == "4":
            print("\n🚪 Exiting the system.")
            break

        else:
            print("\n❌ Invalid choice, please enter a valid option.")

# Run the interactive simulation
if __name__ == "__main__":
    main()



===== Kerberos Authentication System =====
1. Request Ticket Granting Ticket (TGT)
2. Request Service Ticket
3. Access Protected Service
4. Exit


KeyboardInterrupt: Interrupted by user